# Proto1

Kysymyksiä:

1. Onko hybridimalli parempi kuin CF-versio algoritmista?

## Importataan tarvittavat kirjastot

In [1]:
import lightfm

import numpy as np
import pandas as pd

## Määritetään halutut ominaisuudet yrityksille

In [2]:
SELECTED_COMPANY_FEATURES = ['location_region_code', 'industry_code']

## Ladataan raakadata yrityksistä

In [3]:
COMPANIES_RAW = pd \
        .read_csv('data/prod_data_companies_2021_08_22.csv',
                  delimiter='\t',
                  na_values='(null)',
                  dtype={
                      'business_id': 'string',
                      'company_name': 'string',
                      'company_form': 'string',
                      'company_form_code': 'string',
                      'location_region': 'string',
                      'location_region_code': 'string',
                      'location_municipality': 'string',
                      'location_municipality_code': 'string',
                      'industry_code': 'string',
                      'company_status': 'string',
                      'company_status_code': 'string',
                      'personnel_class': 'string'
                  }
                  )

## Käsitellään ja poimitaan halutut yritysdatat

In [4]:
ITEM_IDS = list(COMPANIES_RAW['business_id'].values)

# pitäisi varmaan prefiksoida koodit, jotta pysyvät uniikkeina, kun niitä lisätään
item_feature_labels_tmp = [COMPANIES_RAW[feature].dropna().unique() for feature in SELECTED_COMPANY_FEATURES]

ITEM_FEATURE_LABELS = [item for sublist in item_feature_labels_tmp for item in sublist]

ITEM_FEATURES = [(company['business_id'], 
                  [company[feature] for feature in SELECTED_COMPANY_FEATURES if str(company[feature]) != '<NA>'])
                     for company in COMPANIES_RAW.to_dict(orient='records')]

print(ITEM_FEATURES[0:10])

[('01423486', ['02', '68201']), ('15697971', ['01', '87302']), ('02373820', ['01', '88999']), ('02105471', ['19', '68201']), ('01556668', ['06', '68201']), ('01556668', ['06', '68201']), ('15697971', ['01', '87302']), ('02026351', ['01', '47730']), ('01165149', ['01', '68201']), ('01497530', ['07', '87301'])]


## Ladataan vuorovaikutusdata

In [5]:
interactions_tmp = pd \
    .read_csv('data/interactions_2021_08_19.csv',
             delimiter='\t',
             dtype={
                 'group_id': 'string',
                 'business_id': 'string',
                 'owner': 'string'
             })

# Poistetaan vuorovaikutusdatasta sellaiset y-tunnukset, joita ei löydy kohteista
INTERACTIONS_RAW = interactions_tmp[interactions_tmp.business_id.isin(ITEM_IDS)]

print(interactions_tmp.shape)
print(INTERACTIONS_RAW.shape)

(548198, 3)
(346309, 3)


## Käsitellään vuorovaikutusdata

Toistaiseksi ainakin pidän ryhmää "käyttäjänä". Oletukseni on, että ryhmä yrityksiä on se taso, jolle suosituksia halutaan luoda.

In [6]:
USER_IDS = list(INTERACTIONS_RAW['group_id'].values.unique())

INTERACTIONS = [(interaction['group_id'], interaction['business_id']) 
                for interaction in INTERACTIONS_RAW.to_dict(orient='records')]

print(len(INTERACTIONS))
print(INTERACTIONS[0:10])

346309
[('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01681709'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '15055514'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01876143'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01863991'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '05363070'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01387534'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01372818'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '18348689'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01421229'), ('c2626398-faac-4ff3-b02d-cdc64b50cdaa', '01446661')]


## Luodaan LightFM:n ymmärtämä Dataset-olio

Kysymyksiä:
1. Tuleeko parempia tuloksia identity_featuret päällä vai poissa?
2. Parempi normalisoinnin kanssa vai ilman?

In [7]:
DATASET = lightfm.data.Dataset(user_identity_features=False, item_identity_features=False)

# user_featureja ei ainakaan vielä ole
DATASET.fit(users=USER_IDS, items=ITEM_IDS, item_features=ITEM_FEATURE_LABELS)

ITEM_FEATURES_DS = DATASET.build_item_features(ITEM_FEATURES, normalize=False)

(INTERACTIONS_DS, WEIGHTS_DS) = DATASET.build_interactions(INTERACTIONS)

USER_MAP_DS = DATASET.mapping()[0]
ITEM_MAP_DS = DATASET.mapping()[2]
ITEM_FEATURE_MAP_DS = DATASET.mapping()[3]

# print(ITEM_FEATURE_MAP_DS)
print(USER_MAP_DS)

AttributeError: module 'lightfm' has no attribute 'data'

## Luo LightFM-malli

Kysymyksiä:
1. Mitkä on sopivat hyperparametrit mallille?
2. Erityisesti mikä on paras loss-funktio?
3. Mitkä on parhaat hyperparametrit mallin sovittamiseen?

In [109]:
MODEL = LightFM(loss='warp')

MODEL.fit(INTERACTIONS_DS, item_features=ITEM_FEATURES_DS, epochs=1, verbose=True)

Epoch: 100%|██████████| 1/1 [00:00<00:00,  2.98it/s]


## Tulostetaan esimerkkisuosituksia

In [142]:
USER_ID = 'fe368d6c-ef4d-46c9-ac94-a22eff3ac1f3'

known_positives = [interaction[1] for interaction in INTERACTIONS if interaction[0] == USER_ID]
known_positives_df = COMPANIES_RAW.loc[COMPANIES_RAW['business_id'].isin(known_positives)]

print("User %s:" % USER_ID)
print("Known positives:")
print(known_positives_df[['business_id', 'company_name', 'industry_code', 'location_region_code']].head(10))
    
print("\n\n")

user_id_ds = USER_MAP_DS[USER_ID]
scores = MODEL.predict(user_id_ds, list(ITEM_MAP_DS.values()), item_features=ITEM_FEATURES_DS)

scores_df = pd.DataFrame.from_records(zip(list(ITEM_MAP_DS.keys()), scores), columns=['business_id', 'scores'])
merged_scores_df = pd.merge(scores_df, COMPANIES_RAW, on='business_id')

print("Top hits:")
print(merged_scores_df[['business_id', 'company_name', 'industry_code', 'location_region_code', 'scores']]
      .sort_values('scores', axis=0, ascending=False).head(10))

User fe368d6c-ef4d-46c9-ac94-a22eff3ac1f3:
Known positives:
     business_id                         company_name industry_code  \
198     01591746               Vuoksen Kirjakauppa Oy         47610   
244     11068254                       Rasnel-Hold Oy         68209   
363     09304537              Parikkalan Autorahti Oy         49410   
395     22370715                         Ice Power Oy         43220   
568     17588365           Rakennusvalvonta Toikka Oy         71126   
753     06961267                       Lepojoutsen Oy         55101   
1073    25890126                Saimaan Työpalvelu Oy         78200   
1172    24869536                              JORO OY         49410   
1199    19031093  Asianajotoimisto Heikki Oikkonen Oy         69101   
1484    21289202                  PTu Konsultointi Oy         95110   

     location_region_code  
198                  <NA>  
244                  <NA>  
363                    09  
395                    09  
568               

## Arvioidaan suositusten laatua

In [145]:
(TRAIN, TEST) = random_train_test_split(INTERACTIONS_DS, test_percentage=0.2)